In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Loading the data

In [ ]:
plt.style.use('ggplot')

df = pd.read_csv("../data_raw/benin-malanville.csv", parse_dates=["Timestamp"])
df.head()

# Data Summary & Missing Values

In [ ]:
df.info()
df.describe()

missing = df.isna().sum()
missing_percent = (missing/len(df))*100
missing_percent.sort_values()


In [ ]:
print("An overview of the dataset: the first 5 rows")
df.head()

In [ ]:
print("An overview of the dataset: the last 5 rows")
df.tail()

In [ ]:
print("An overview of the dataset: a random sample of 5 rows")
df.sample(5)

In [ ]:
print("An overview of the dataset: the shape of the dataset")
print(df.info())

In [ ]:
print("Check for missing values:")
print(df.isna().sum())

In [ ]:
print("The description of the numeric columns:")
print(df.describe())

In [ ]:
print("The description of all columns:")
print(df.describe(include='all'))

# Outlier Detection (Z-Score)

In [ ]:
columns_to_check = ['GHI','DNI','DHI','ModA','ModB','WS','WSgust']

z = np.abs(stats.zscore(df[columns_to_check], nan_policy="omit"))
outlier_rows = np.where(z > 3)[0]
len(outlier_rows)


# Clean Missing Values

In [ ]:
df[columns_to_check] = df[columns_to_check].fillna(df[columns_to_check].median())
df_clean = df[(z < 3).all(axis=1)]
df_clean.head()


# Export Cleaned Data

In [ ]:
df_clean.to_csv("../data/benin_clean.csv", index=False)


# Univariate Analysis

In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in num_cols:
    plt.figure()
    sns.histplot(df[c], kde=True)
    plt.title(f"Distribution: {c}")
    plt.xlabel(c)
    plt.ylabel("Count")
    plt.show()


##  Univariate Analysis – Interpretation

### Solar Radiation Variables — GHI, DNI, DHI

- Most values are zero or near zero at night → 🕛 expected behavior

- Skewed heavily to the right → only peak around midday 🌞

- Peak daytime GHI approx. 600–1000 W/m² suggests good sunlight

- Interpretation example:

- Solar irradiance values are right-skewed because radiation exists only during daytime. Higher peaks indicate strong solar availability in Benin.

### Module Sensor Readings — ModA & ModB

- Similar distribution to GHI 

- If ModB < ModA often → soiling slowly reduces ModB performance

- Module readings show operational consistency but appear affected by soiling, visible as slightly lower values compared to irradiance.

### Ambient Temperature — Tamb

- Bell-shaped (normal-like) with peak around 25–35°C

- High daytime temperature conditions 🌡️ — typical for West Africa

- Higher module temperature may reduce solar panel efficiency.

### Relative Humidity — RH

- More spread → can be high in mornings/evenings

- Inverse relationship with temperature

- High humidity periods may reduce irradiance due to atmospheric scattering.

### Wind Speed — WS & WSgust

- Mostly low wind speed (0–5 m/s)

- Occasional gusts → outliers visible

- Low wind → Some heat retention on modules, contributing to higher temperatures.

### Barometric Pressure — BP

- Normally distributed → expected environmental stability

### Precipitation

- Mostly zero 

- Some small values → occasional rainfall

- Cleaning by natural rainfall is occasional — proper cleaning needed.

### 🔍 Summary

- Most meteorological and irradiance features show right-skewed distributions due to natural daily solar cycles. Temperature follows a more normal distribution indicating climate consistency in Benin. Wind, humidity, and precipitation distributions reveal the environmental conditions affecting solar panel performance.



## Correlation Matrix & Heatmap (numeric)

In [ ]:
sns.heatmap(df_clean[['GHI','DNI','DHI','TModA','TModB','RH']].corr(), annot=True)
plt.show()


## Correlation Heatmap Insights

### Strong Positive Correlations
	
- GHI ↔ DNI	
High total irradiance occurs when sunlight is direct and skies are clear → stronger solar generation potential

- ModA & ModB ↔ GHI	
Both modules produce more power with higher sunlight levels → confirming good sensor and system response

- Tamb ↔ GHI	
Ambient temperature increases as sunlight intensity rises, especially during midday

### Operational Insight:

- Good irradiance conditions directly increase power generation, but also raise ambient temperature — which may reduce panel efficiency due to overheating.

### Negative Correlations

- RH ↔ Tamb	
As temperature rises, relative humidity drops → hot and dry midday conditions typical of Benin

- RH ↔ Irradiance (GHI/DNI)	
Strong sunlight rapidly dries the air → humidity lowest during maximum energy production periods

### Operational Insight:

- Low humidity supports better sunlight transmission, but combined with high heat may increase thermal losses in panels.

### Final Interpretation

The correlation patterns confirm that Benin has a strong solar resource, driven by high irradiance and clear-sky conditions. However, the combination of high temperatures and low cooling wind may negatively affect photovoltaic efficiency. This suggests that cooling strategies, ventilation, or panel material selection are important considerations for maximizing output in Benin’s climate.

# Box Plots (numeric)

In [ ]:
for c in num_cols:
    plt.figure()
    plt.boxplot(df[c].dropna(), vert=True)
    plt.title(f"Box plot: {c}")
    plt.ylabel(c)
    plt.show()


##  Box Plot Analysis – Interpretation

### Solar Radiation Variables (GHI, DNI, DHI)

- Large concentration of values near zero → representing night-time when solar radiation is absent 

- Upper whiskers extend to high values → strong solar exposure during daytime 

- Presence of high-value outliers around noon hours → possible cloud effects or temporary irradiance spikes

   Interpretation:

- Solar irradiance distributions reveal a typical daily solar cycle with high variability during daylight. Outliers mostly reflect natural atmospheric changes and should not be removed blindly.

### Module Performance (ModA & ModB)

- Similar distribution to GHI but slightly lower median values

- More outliers in ModB → likely due to soiling effects or sensor drift

  Interpretation:

Module B appears more affected by soiling, showing reduced and inconsistent readings compared to Module A. Cleaning events likely restore performance.

### Ambient Temperature (Tamb)

- Higher median around 28–35°C

- Moderate number of upper outliers → extreme midday heat

   Interpretation:

- High panel temperatures in Benin could negatively impact photovoltaic efficiency, highlighting importance of cooling and airflow.

### Relative Humidity (RH)

- Wider box showing high variability

- Outliers mostly at very low humidity levels

   Interpretation:

- Humidity fluctuates significantly throughout the day; lower humidity generally coincides with hotter midday hours.

### Wind Speed (WS) & Wind Gust (WSgust)

- Majority of values clustered near 0–3 m/s

- Long upper whiskers with several outliers → momentary gust activity 🍃

  Interpretation:

- Benin is generally low-wind, possibly reducing natural cooling of panels, but occasional gusts help dissipate heat.

### Barometric Pressure (BP)

- Very few outliers

- Stable range → consistent weather patterns

  Interpretation:

- Atmospheric pressure is largely stable, supporting steady solar generation conditions.


## Bivariate Analysis
 Scatter Plots (selected pairs, colored by species)

In [ ]:

sns.scatterplot(data=df, x="GHI", y="ModB")
plt.title("GHI vs ModB")
plt.show()

plt.figure(figsize=(7,5))
sns.scatterplot(data=df, x="WS", y="GHI")
plt.title("Wind Speed vs GHI")
plt.show()

## Bivariate Analysis – Interpretation 
### Solar Radiation Relationship Insights (GHI vs DNI vs DHI)

- Strong positive relationship between GHI and DNI during clear sunlight conditions

- DHI increases when clouds scatter sunlight, causing drop in DNI

- Radiation metrics follow expected atmospheric behavior patterns

   Interpretation:

- Direct sunlight contributes most to total solar gain in Benin. Cloudy periods show reduced DNI but increased diffuse radiation (DHI).

### Temperature vs Solar Performance (GHI vs Tamb / RH)

- Scatter plots & trend lines show:

- Tamb increases as GHI increases → peak heat at midday 

- RH decreases as temperature and GHI increase

  Interpretation:

- Higher sunlight increases temperature, while humidity drops because heat evaporates moisture. High panel temperature can reduce solar module efficiency.

### Module Power Relationship (ModA & ModB vs GHI)

- Both ModA and ModB rise with increasing GHI

- ModA consistently performs slightly higher

- Performance gap widens under dusty/soiled conditions

  Interpretation:

- Sensor B (ModB) shows reduced output when soiled — confirming soiling reduces system efficiency. Cleaning cycles likely restore performance.

### Wind Influence (WS / WSgust vs GHI)

- Weak relationship between wind speed and irradiance

- Slight increase in wind around high-irradiance periods → daytime convection effect

- However, wind speeds are mostly low (< 3 m/s)

 Interpretation:

- Low wind limits natural cooling of modules, potentially increasing overheating losses in Benin.

### Summary

- high temperature and low cooling wind may reduce panel efficiency, while soiling causes measurable degradation in ModB sensor performance. Understanding these relationships supports optimal site design and maintenance planning.

## Multivariate Analysis
 Pairplot (quick overview)

In [ ]:
# Pairplot for core features
pairplot_cols = ["GHI", "DNI", "DHI", "ModA", "ModB", "Tamb", "RH"]

sns.pairplot(df[pairplot_cols], diag_kind="kde")
plt.suptitle("Pairplot - Multivariate Relationships (Benin Dataset)", y=1.02)
plt.show()


## Multivariate Analysis – Interpretation 

### Solar Irradiance vs Module Output

- Strong linear relationships between GHI, DNI, and both ModA & ModB

- This confirms that the modules respond directly and accurately to solar irradiance.

- ModA consistently performs slightly better than ModB → suggests soiling or sensor degradation on ModB.

  Insight:

- Regular cleaning is important to maintain performance and reduce soiling losses.
###  Temperature Effects

- Tamb increases as solar irradiance increases

- Module temperatures (TModA & TModB, if included) follow the same behavior

 Insight:

- High heat during peak irradiance may reduce efficiency due to thermal stress on panels.

### Humidity Behavior

- RH decreases when irradiance and temperature rise

- A clear inverse relationship is present in multiple plots

 Insight:

- Dry and sunny conditions are best for solar production but may accelerate equipment wear and dust accumulation.

### Wind Impacts (from earlier bivariate analysis)

- Wind speed shows very weak relationships with irradiance

- Low wind conditions reduce natural cooling, increasing the effect of panel heating

 Insight:

- Proper spacing and airflow design are essential in Benin’s solar installations.

### Final Summary

Multivariate analysis reveals that Benin has strong solar energy generation potential with clear-sky conditions dominating the irradiance profile. However, high ambient temperatures and low wind speeds can negatively impact module performance by reducing cooling efficiency. Soiling also plays a role in reducing performance for certain sensors or modules.

# Cleaning Impact on Sensor Output

In [ ]:
df_clean.groupby("Cleaning")[['ModA','ModB']].mean().plot(kind='bar')
plt.title("Effect of Cleaning on Module Performance")
plt.show()